In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2"
import ray

ray.init(ignore_reinit_error=True, num_gpus=2)

from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd

train_data = TabularDataset('/home/zhenghao/kaggle/train.csv')
test_data = TabularDataset( '/home/zhenghao/kaggle/test.csv')

params = {
    
    "XGB": {
        
        "colsample_bytree": 0.516652313273348,
        "enable_categorical": True,
        "learning_rate": 0.015,
        "max_depth": 9,
        "min_child_weight": 0.8567068904025429,
        "max_bin": 262143,
        "n_estimators": 10000,
        
          },
    
    'GBM': {
          
        'n_estimators': 1190,
        'learning_rate': 0.22952000374471332,
        'max_depth': 13, 
        'reg_alpha': 8.200152384535924,
        'reg_lambda': 4.285393733702208, 
        'num_leaves': 100, 
        'subsample': 0.6497981764924947, 
        'colsample_bytree': 0.37368304607248115
           
           } 

}
included_models = ["GBM", "XGB",]


NVMLError_Unknown: Unknown Error

In [19]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import pynvml
pynvml.nvmlInit()

def check_gpu_status():
    device_count = pynvml.nvmlDeviceGetCount()
    for i in range(device_count):
        handle = pynvml.nvmlDeviceGetHandleByIndex(i)
        print(f"GPU {i}: {pynvml.nvmlDeviceGetName(handle)}, Memory: {pynvml.nvmlDeviceGetMemoryInfo(handle).used / 1024**2} MB used")

try:
    check_gpu_status()
except pynvml.NVMLError as e:
    print(f"NVML Error: {e}")

pynvml.nvmlShutdown()


NVML Error: Unknown Error


In [ ]:
predictor = TabularPredictor(label='Response').fit(
    train_data=train_data,
    hyperparameters = params,
        
    presets='best_quality',
    dynamic_stacking=False, 
    included_model_types = included_models,)
predictions = predictor.predict(test_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240705_043605"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #46-Ubuntu SMP Fri Jul 10 00:24:02 UTC 2020
CPU Count:          48
Memory Avail:       226.81 GB / 251.56 GB (90.2%)
Disk Space Avail:   1819.83 GB / 3509.42 GB (51.9%)
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20240705_043605"
Train Data Rows:    11504798
Train Data Columns: 11
Label Column:       Response
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the pro

AssertionError: Trainer has no fit models that can infer.

In [ ]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.877677,accuracy,0.304627,6197.449034,0.013403,3.900088,2,True,14
1,NeuralNetTorch,0.877642,accuracy,0.268633,6163.338346,0.268633,6163.338346,1,True,12
2,RandomForestGini,0.877582,accuracy,0.239415,343.882004,0.239415,343.882004,1,True,5
3,RandomForestEntr,0.877477,accuracy,0.265070,429.444768,0.265070,429.444768,1,True,6
4,NeuralNetFastAI,0.877147,accuracy,0.730139,4127.311939,0.730139,4127.311939,1,True,10
5,ExtraTreesGini,0.877008,accuracy,0.221422,209.560388,0.221422,209.560388,1,True,8
6,ExtraTreesEntr,0.877008,accuracy,0.231594,230.594307,0.231594,230.594307,1,True,9
7,CatBoost,0.876999,accuracy,0.022590,30.210600,0.022590,30.210600,1,True,7
8,LightGBMXT,0.876999,accuracy,0.024895,8.225573,0.024895,8.225573,1,True,3
9,LightGBMLarge,0.876999,accuracy,0.045202,9.100466,0.045202,9.100466,1,True,13


In [ ]:
d_s = pd.read_csv('/home/zhenghao/kaggle/sample_submission.csv')

In [ ]:
%%time

Y_T_PR_PROBA = predictor.predict_proba(test_data)

d_s['Response'] = Y_T_PR_PROBA.iloc[:, 1]

d_s.to_csv('Submission_AutoGluon_.csv', index=False)
d_s.head()

CPU times: user 5min 36s, sys: 2.74 s, total: 5min 39s
Wall time: 37 s


,id,Response
0,11504798,0.047402
1,11504799,0.307118
2,11504800,0.330708
3,11504801,0.034150
4,11504802,0.069136
